# XGBoost

# Model

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [3]:
!pip install xgboost

In [4]:
import xgboost as xgb

#### XGBoost dökümanında xgb geliştiricisi pandas dataframe'i veya numpy array'i yerine xgb'nin kendi veri yapısı kullanılırsa daha performanslı sonuçlar elde edileceğini söylüyor.

In [8]:
DM_train = xgb.DMatrix(data = X_train, label = y_train)
DM_test = xgb.DMatrix(data = X_test, label = y_test)
# xgb'nin kendi veri yapısını oluşturduk
# ama uygularken bu veri yapısının kullanılması zorunlu değil!

In [9]:
from xgboost import XGBRegressor

In [13]:
xgb_model = XGBRegressor().fit(X_train, y_train)

# Tahmin

In [16]:
y_pred = xgb_model.predict(X_test)

In [17]:
np.sqrt(mean_squared_error(y_test, y_pred))

355.46515176059927

# Model Tuning

In [18]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

 booster ağaca dayalı bir booster olacak
 
 colsample_bytree ağaç başına değişkenlerden alınacak olan örnek oranı
 
 learning_rate daraltma adım boyu olarak geçer, overfittingi engellemek için kullanıdğımız değişkenlerden birisidir. 
 0-1 arası değer olur, çok küçük olması tahmin süresini uzatır ama başarı oranını da arttırır.
 
 max_depth overfittingi engellemek için kullanıyoruz
 
 n_estimators = kullanılacak ağaç sayısı gibi
 

In [19]:
xgb_grid = {
     'colsample_bytree': [0.4, 0.5,0.6,0.9,1], 
     'n_estimators':[100, 200, 500, 1000],
     'max_depth': [2,3,4,5,6],
     'learning_rate': [0.1, 0.01, 0.5]
}

In [60]:
xgb = XGBRegressor()

xgb_cv = GridSearchCV(xgb, 
                      param_grid = xgb_grid, 
                      cv = 10, 
                      n_jobs = -1,
                      verbose = 2)


xgb_cv.fit(X_train, y_train)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=200; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=500; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=2, n_estimators=1000; total time=   0.2s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=3, n_estimators=200; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=3, n_estimators=200; total time=   0.1s
[CV] END colsample_bytree=0.4, learning_rate=0.1, max_depth=3, n_estimators=500; total time=   0.1s
[CV] END colsample_bytree=0.4, lea

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_ca...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                  

In [61]:
xgb_cv.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 1000}

In [62]:
# final model

In [74]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.5, 
                         learning_rate = 0.1, 
                         max_depth = 2, 
                         n_estimators = 1000) 

xgb_tuned = xgb_tuned.fit(X_train,y_train) # best_params_ le çalıştırınca olan sonuç

In [75]:
y_pred = xgb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

357.4251457287713

In [76]:
# NEDEN ALTTAKI PARAMETRELERDE HATA DEGERI DAHA DUSUK OLUYOR?

In [81]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.9, 
                         learning_rate = 0.01, 
                         max_depth = 2, 
                         n_estimators = 1000) 

xgb_tuned = xgb_tuned.fit(X_train,y_train)

In [82]:
y_pred = xgb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

346.8615157484009

## Gelişmiş Hiperparametre Optimizasyonu

Modellerimizi nasıl daha iyi hiperparametre optimizasyonun tabi tutarız ?

Gelişmiş hiperparametre optimizasyonu konusunda yaygın görüş şudur:

1. Modelimizin önemli olarak kullanıdğı parametreler ve önem sıraları belirlenir. (bizim problemlerde gridsearch'te kullandıklarımız.)

2. Önem sırası belirlendikten sonra parametrelerin ön tanımlı değerleri sabit bırakılır, öncelikle en önemli parametrenin değerleri uzayda aranır (vektörel)

örn: mesela yukarda learning_rate önce bulunur öbürleri sabit tutularak, sonra learning_rate sabitlenip max_depth aranır vesaire ve böyle 3-4 iterasyon sonrası birbirinden farklı adımlarda ve son olarak da bunları birbirleriyle çarpıştırarak genel durumu gözlemleyecek şekilde hiperparametre optimizasyonu denenir. 

### önem düzeyleri neye göre belirlenir ??????